In [1]:
import pandas as pd 
import networkx as nx
import matplotlib.pyplot as plt
from pathlib import Path
import graspologic as gl
from celegans_connectome_analysis.get_nx import get_adult_c_elegans_nx,get_fxnl_nx

In [2]:
G_fx = get_fxnl_nx()
G_stx = get_adult_c_elegans_nx()

/home/gwyneth-local/celegans_connectome_analysis/celegans_connectome_analysis/get_nx.py:28: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["neuron type"] = df["neuron type"].fillna(method='ffill')


In [3]:
# goal: make a di_graph that contants functional and structural information
fx_nodes = set(G_fx.nodes())
stx_nodes = set(G_stx.nodes())

print(stx_nodes - fx_nodes)
print(fx_nodes - stx_nodes)

{'BDUL', 'BWM-DR07', 'GLRL', 'CEPshVL', 'PVT', 'BWM-VR06', 'BWM-DR02', 'RIFL', 'BWM-VR01', 'BWM-DR06', 'PVR', 'ALNL', 'GLRDL', 'HSNL', 'BWM-DR01', 'DVC', 'BWM-VL06', 'SIADR', 'PVQL', 'GLRR', 'PVQR', 'SIBVL', 'CEPshDR', 'BDUR', 'AIAR', 'PVCR', 'BWM-DL07', 'BWM-VR07', 'BWM-DL04', 'ADFR', 'GLRVR', 'BWM-DR05', 'BWM-VR08', 'SDQR', 'BWM-VL07', 'PLNL', 'SIADL', 'BWM-DL01', 'CEPshVR', 'IL1L', 'SDQL', 'BWM-VR05', 'SIAVR', 'AIAL', 'ALNR', 'BWM-VL05', 'AVM', 'SIAVL', 'PVNL', 'BWM-DL03', 'PVNR', 'BWM-DL06', 'AVFR', 'RIPR', 'BWM-DR03', 'BWM-VL01', 'PVPR', 'CANR', 'CANL', 'BWM-VL08', 'BWM-VL04', 'BWM-VR04', 'HSNR', 'PVCL', 'ALMR', 'ALML', 'BWM-VL03', 'PVPL', 'RMHL', 'PLNR', 'SIBDL', 'BWM-VR03', 'SIBDR', 'BWM-VR02', 'RMHR', 'CEPshDL', 'GLRDR', 'BWM-VL02', 'DVA', 'BWM-DL08', 'IL2VR', 'BWM-DL05', 'BWM-DL02', 'GLRVL', 'BWM-DR08', 'BWM-DR04'}
{'I2L', 'M1', 'MCL', 'AS1', 'SABVL', 'IL2V4', 'I1L', 'VB1', 'M2R', 'MI', 'SABD', 'VA1', 'I6', 'I1R', 'NSMR', 'NSML', 'DB2', 'DD1', 'M2L', 'SABVR', 'I2R', 'AVG', 'I3

In [33]:
def combined_graph(stx, fx):
    G = nx.DiGraph()
    transfer_info(G, fx, 'fx_weight')
    transfer_info(G, stx, 'stx_weight')
    return G

def transfer_info(G: nx.DiGraph, source: nx.DiGraph, weight_label: str):
    for node in source.nodes():
        if node not in G.nodes():
            G.add_node(node)
        else:
            if 'neuron type' not in G.nodes[node]:
                print("Have not set neuron type for node")
            else:
                print(f"Node matches neuron_type already in G: {(G.nodes[node]['neuron type'] == source.nodes[node]['neuron type'])}")
        
        G.nodes[node]['neuron type'] = source.nodes[node]['neuron type']
        # add out edge information
        for target, infos in source[node].items():
            if not G.has_edge(node, target):
                G.add_edge(node, target)
            renamed_infos = {weight_label: infos['weight']}
            G[node][target].update(renamed_infos)

In [34]:
G = combined_graph(G_stx, G_fx)
print(G)

Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set neuron type for node
Have not set n

In [4]:
# mini tests

# test 1: check if all fx nodes are in G
def check_fx(G: nx.DiGraph, fx_nodes: set):
    for node in fx_nodes:
        assert node in G.nodes(), f"Node {node} not in G"

# test 2: check if all stx nodes are in G
def check_stx(G: nx.DiGraph, stx_nodes: set):
    for node in stx_nodes:
        assert node in G.nodes(), f"Node {node} not in G"

# test 3: check if all fx edges are in G and weights match
def check_fx_edges(G: nx.DiGraph, fx: nx.DiGraph):
    for node in fx.nodes():
        for target, infos in fx[node].items():
            assert G[node][target]['fx_weight'] == infos['weight'], f"Weight mismatch between G and fx"

# test 4: check if all stx edges are in G and weights match
def check_stx_edges(G: nx.DiGraph, stx: nx.DiGraph):
    for node in stx.nodes():
        for target, infos in stx[node].items():
            assert G[node][target]['stx_weight'] == infos['weight'], f"Weight mismatch between G and stx"

# test 5: check if all nodes have stx neuron type attribute
def check_neuron_type(G: nx.DiGraph, stx: nx.DiGraph):
    for node in stx.nodes():
        assert G.nodes[node]['neuron type'] == stx.nodes[node]['neuron type'], f"Neuron type mismatch between G and stx"

In [36]:
# run tests
check_fx(G, fx_nodes)
check_stx(G, stx_nodes)
check_fx_edges(G, G_fx)
check_stx_edges(G, G_stx)
check_neuron_type(G, G_stx)

In [37]:
# save graph
nx.write_gpickle(G, 'combined_graph.gpickle')

In [5]:
# load and test
G_loaded = nx.read_gpickle('combined_graph.gpickle')
check_fx(G_loaded, fx_nodes)
check_stx(G_loaded, stx_nodes)
check_fx_edges(G_loaded, G_fx)
check_stx_edges(G_loaded, G_stx)
check_neuron_type(G_loaded, G_stx)